In [2]:
import nest_asyncio,os
nest_asyncio.apply()

In [4]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from dotenv import load_dotenv
load_dotenv("D:\\Code\\AI\\.env")

True

Helper Function

In [15]:
from pydantic import BaseModel


def to_markdown(data, indent=0):
    markdown = ""
    if isinstance(data, BaseModel):
        data = data.model_dump()
    if isinstance(data, dict):
        for key, value in data.items():
            markdown += f"{'#' * (indent + 2)} {key.upper()}\n"
            if isinstance(value, (dict, list, BaseModel)):
                markdown += to_markdown(value, indent + 1)
            else:
                markdown += f"{value}\n\n"
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list, BaseModel)):
                markdown += to_markdown(item, indent)
            else:
                markdown += f"- {item}\n"
        markdown += "\n"
    else:
        markdown += f"{data}\n\n"
    return markdown

# Agent Class Components

The Agent class can be thought of as a container for the following components, which define its behavior and capabilities:

| Component                   | Description                                                                                                                                                                  |
| --------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **System Prompt(s)**        | A set of instructions, crafted by the developer, that guide the Large Language Model (LLM) in its operation.                                                              |
| **Function Tool(s)**       | Functions that the LLM has the ability to invoke during response generation. These tools provide the LLM with external information or capabilities.                                             |
| **Structured Result Type** | The specific data type the LLM is required to return as the final output of its run, if one is defined.                                                                            |
| **Dependency Type Constraint** | Defines the dependencies that can be used by System prompts functions, tools, and result validators when they are executed.                                                                         |
| **LLM Model**               | An optional default Large Language Model (LLM) associated with the agent.  A different model can also be specified when the agent is run.                         |
| **Model Settings**          | Optional default configuration parameters used to fine-tune the behavior of the model requests. These can be overridden when running the agent. |


This example demonstrates the basic usage of PydanticAI agents.
Key concepts:
- Creating a basic agent with a system prompt
- Running synchronous queries
- Accessing response data, message history, and costs

#### 1. Simple Agent

In [ ]:
# --------------------------------------------------------------
# 1. Simple Agent - Hello World Example
# --------------------------------------------------------------


deepseek_model = OpenAIModel(
    model_name='deepseek-chat',
    provider=OpenAIProvider(base_url='https://api.deepseek.com/',
                            api_key=os.getenv("DEEPSEEK_API_KEY"))
    )

agent = Agent(model=deepseek_model,system_prompt=" You are a helpful assistant.")

response=agent.run_sync("What is the capital of the Pandavas?")

print(response.data)
print(response.all_messages())
# print(response.cost()) #AttributeError: 'AgentRunResult' object has no attribute 'cost'


The capital of the Pandavas, the legendary heroes of the Indian epic Mahabharata, was **Indraprastha**. According to the epic, the city was established by the Pandavas after they were granted a barren land by their cousins, the Kauravas. With the help of Lord Krishna and the divine architect Vishwakarma, they transformed the land into a magnificent city, which became their capital. Indraprastha is often identified with modern-day Delhi, India.
[ModelRequest(parts=[SystemPromptPart(content=' You are a helpful assistant.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='What is the capital of the Pandavas?', timestamp=datetime.datetime(2025, 3, 16, 15, 50, 58, 392552, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content='The capital of the Pandavas, the legendary heroes of the Indian epic Mahabharata, was **Indraprastha**. According to the epic, the city was established by the Pandavas after they were grant

In [10]:
response2 = agent.run_sync(user_prompt="What was my 2 previous question? ",
                           message_history=response.new_messages(),)
print(response2.data)

Your two previous questions were:

1. **"What is the capital of the Pandavas?"**
2. **"What was my 2 previous question?"**


#### 2. Agent with Structured Response

This example shows how to get structured, type-safe responses from the agent.
Key concepts:
- Using Pydantic models to define response structure
- Type validation and safety
- Field descriptions for better model understanding

In [13]:
# --------------------------------------------------------------
# 2. Agent with Structured Response
# --------------------------------------------------------------

from pydantic import BaseModel,Field

class GetResponse(BaseModel):
    """ Structured Response with metadata"""
    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")

# Model is configure above
agent2=Agent(model=deepseek_model,system_prompt=""" You are a intelligent customer support agent. Analyze queries carefully and provide structured response""",
             result_type=GetResponse)

response = agent2.run_sync(" How can I track my order #12345")
print(response.data.model_dump_json(indent=2))

{
  "response": "To track your order #12345, please visit our website and go to the 'Order Tracking' section. Enter your order number and the email address used for the purchase. You will receive real-time updates on the status of your order. If you encounter any issues, feel free to contact our support team.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral"
}


#### 3. Agent with Structured Response & Dependencies

This example demonstrates how to use dependencies and context in agents.
Key concepts:
- Defining complex data models with Pydantic
- Injecting runtime dependencies
- Using dynamic system prompts

In [16]:
from typing import List,Optional
from pydantic_ai import Agent,ModelRetry,RunContext,Tool


# Define order schema
class Order(BaseModel):
    """ Structure for order details"""
    order_id: str
    status: str
    items : List[str]

# Define Customer schema
class CustomerDetails(BaseModel):
    """Structure for incoming customer queries"""
    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] =None

class GetResponse(BaseModel):
    """ Structured Response with metadata"""
    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")

# Agent with structured output and dependencies

support_agent= Agent(model=deepseek_model,
                     result_type=GetResponse,
                     deps_type=CustomerDetails,
                     retries=3,
                     system_prompt=""" You are an intelligent customer support agent.
                                    Analyze queries carefully and provide structured response.
                                    Always greet the customer and provide a helpful response.""")

# Add dynamic system prompt based on dependencies
@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails])-> str:
    return f"Customer details: {to_markdown(ctx.deps)}" # These depend in some way on the context that is'nt know until runtime

In [20]:
customer = CustomerDetails(
    customer_id="1",
    # customer_id=1,
    name="John Doe",
    email="john.doe@example.com",
    orders=[
        Order(order_id="12345", status="shipped", items=["Blue Jeans", "T-Shirt"]),
    ],
)

response = support_agent.run_sync(user_prompt="What did I order?", deps=customer)

response.all_messages()
print(response.data.model_dump_json(indent=2))

print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)


{
  "response": "You ordered the following items: Blue Jeans and T-Shirt.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral"
}
Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
You ordered the following items: Blue Jeans and T-Shirt.

Status:
Follow-up Required: False
Needs Escalation: False
